In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
fire_dataset_path = '/content/drive/MyDrive/Forest Fire Project/Datasets/FIRE Dataset'
wildfire_dataset_path = '/content/drive/MyDrive/Forest Fire Project/Datasets/Wildfire Detection Image Data'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import time
import onnx

In [4]:
# Define image transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)  # Normalize to [-1, 1]
])

# Load FIRE Dataset
fire_train = datasets.ImageFolder(root=f"{fire_dataset_path}/train", transform=transform)
fire_val = datasets.ImageFolder(root=f"{fire_dataset_path}/val", transform=transform)
fire_test = datasets.ImageFolder(root=f"{fire_dataset_path}/test", transform=transform)

# Load Wildfire Dataset
wildfire_train = datasets.ImageFolder(root=f"{wildfire_dataset_path}/train", transform=transform)
wildfire_val = datasets.ImageFolder(root=f"{wildfire_dataset_path}/val", transform=transform)
wildfire_test = datasets.ImageFolder(root=f"{wildfire_dataset_path}/test", transform=transform)

# Create DataLoaders
batch_size = 32

fire_train_loader = DataLoader(fire_train, batch_size=batch_size, shuffle=True)
fire_val_loader = DataLoader(fire_val, batch_size=batch_size, shuffle=False)
fire_test_loader = DataLoader(fire_test, batch_size=batch_size, shuffle=False)

wildfire_train_loader = DataLoader(wildfire_train, batch_size=batch_size, shuffle=True)
wildfire_val_loader = DataLoader(wildfire_val, batch_size=batch_size, shuffle=False)
wildfire_test_loader = DataLoader(wildfire_test, batch_size=batch_size, shuffle=False)


## Custom 3 layer CNN model

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 2)  # Binary classification (fire or not)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [B, 16, 112, 112]
        x = self.pool(F.relu(self.conv2(x)))  # [B, 32, 56, 56]
        x = self.pool(F.relu(self.conv3(x)))  # [B, 64, 28, 28]
        x = x.view(-1, 64 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [5]:
from torchsummary import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomCNN().to(device)

summary(model, (3, 224, 224))  # Shows parameters, shapes


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]             448
         MaxPool2d-2         [-1, 16, 112, 112]               0
            Conv2d-3         [-1, 32, 112, 112]           4,640
         MaxPool2d-4           [-1, 32, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          18,496
         MaxPool2d-6           [-1, 64, 28, 28]               0
            Linear-7                  [-1, 128]       6,422,656
            Linear-8                    [-1, 2]             258
Total params: 6,446,498
Trainable params: 6,446,498
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 13.40
Params size (MB): 24.59
Estimated Total Size (MB): 38.57
----------------------------------------------------------------


In [4]:
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim

def train_model(model, train_loader, val_loader, epochs=10, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
        for inputs, labels in loop:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            loop.set_postfix(loss=loss.item(), acc=100. * correct / total)

        val_acc = evaluate_model(model, val_loader)
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {running_loss/len(train_loader):.4f}, Val Acc: {val_acc:.2f}%")

    print("Training complete.")
    return model


def evaluate_model(model, loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    return 100 * correct / total


In [5]:
model = CustomCNN()
trained_model = train_model(model, fire_train_loader, fire_val_loader, epochs=10, lr=0.001)

Epoch [1/10], Train Loss: 0.3576, Val Acc: 88.75%


Epoch [2/10], Train Loss: 0.1568, Val Acc: 91.25%


Epoch [3/10], Train Loss: 0.1263, Val Acc: 91.25%


Epoch [4/10], Train Loss: 0.0772, Val Acc: 97.50%


Epoch [5/10], Train Loss: 0.0443, Val Acc: 95.00%


Epoch [6/10], Train Loss: 0.0348, Val Acc: 97.50%


Epoch [7/10], Train Loss: 0.0151, Val Acc: 96.25%


Epoch [8/10], Train Loss: 0.0079, Val Acc: 97.50%


Epoch [9/10], Train Loss: 0.0037, Val Acc: 95.00%


Epoch [10/10], Train Loss: 0.0015, Val Acc: 96.25%
Training complete.


In [6]:
test_accuracy = evaluate_model(trained_model, fire_test_loader)
print(f"🔥 Test Accuracy on FIRE Dataset: {test_accuracy:.2f}%")

🔥 Test Accuracy on FIRE Dataset: 95.20%


In [7]:
!pip install ptflops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [8]:
from ptflops import get_model_complexity_info

with torch.cuda.device(0):
    macs, params = get_model_complexity_info(CustomCNN(), (3, 224, 224), as_strings=True,
                                             print_per_layer_stat=True, verbose=True)
    print(f'FLOPs: {macs}')
    print(f'Params: {params}')


CustomCNN(
  6.45 M, 100.000% Params, 146.51 MMac, 98.118% MACs, 
  (conv1): Conv2d(448, 0.007% Params, 22.48 MMac, 15.054% MACs, 3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(4.64 k, 0.072% Params, 58.2 MMac, 38.978% MACs, 16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(18.5 k, 0.287% Params, 58.0 MMac, 38.844% MACs, 32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(0, 0.000% Params, 1.4 MMac, 0.941% MACs, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(6.42 M, 99.630% Params, 6.42 MMac, 4.301% MACs, in_features=50176, out_features=128, bias=True)
  (fc2): Linear(258, 0.004% Params, 258.0 Mac, 0.000% MACs, in_features=128, out_features=2, bias=True)
)
FLOPs: 149.32 MMac
Params: 6.45 M


In [10]:
import torch

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [11]:
model = trained_model.to(device).eval()
dummy_input = torch.randn(1, 3, 224, 224).to(device)

In [12]:
# Warm-up
for _ in range(10):
    _ = model(dummy_input)

# Measure
import time
start = time.time()
for _ in range(100):
    _ = model(dummy_input)
end = time.time()

avg_time_ms = (end - start) / 100 * 1000
print(f"⏱️ Avg inference time per image: {avg_time_ms:.2f} ms")

⏱️ Avg inference time per image: 0.53 ms


Export CNN model

In [14]:
# Assuming 'model' is your trained model
torch.save(model.state_dict(), "custom_cnn_fire.pth")

In [16]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 110.3 MB/s eta 0:00:00


In [17]:
import torch

# Make sure model is on CPU for export or on the same device as dummy input
model = CustomCNN()
model.load_state_dict(torch.load("custom_cnn_fire.pth"))
model.eval()

# Dummy input (batch size 1, 3 color channels, 224x224 image)
dummy_input = torch.randn(1, 3, 224, 224)

# Export to ONNX
torch.onnx.export(
    model,
    dummy_input,
    "custom_cnn_fire.onnx",
    input_names=['input'],
    output_names=['output'],
    opset_version=11
)

print("✅ Exported as custom_cnn_fire.onnx — ready for Netron!")

✅ Exported as custom_cnn_fire.onnx — ready for Netron!


## ResNet-18 Model

Colab crashed re adding dependencies

In [5]:
model = models.resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, 2)  # Adjusting the final layer for 2 classes

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
def train_model(model, train_loader, val_loader, epochs=10, lr=0.001):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_acc = evaluate_model(model, val_loader)
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {running_loss/len(train_loader):.4f}, Val Acc: {val_acc:.2f}%")

    print("Training complete.")
    return model

def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total


In [9]:
trained_model = train_model(model, fire_train_loader, fire_val_loader, epochs=10, lr=0.001)

Epoch [1/10], Train Loss: 0.2406, Val Acc: 72.50%
Epoch [2/10], Train Loss: 0.9308, Val Acc: 50.00%
Epoch [3/10], Train Loss: 0.3574, Val Acc: 87.50%
Epoch [4/10], Train Loss: 0.2588, Val Acc: 95.00%
Epoch [5/10], Train Loss: 0.1480, Val Acc: 80.00%
Epoch [6/10], Train Loss: 0.1324, Val Acc: 95.00%
Epoch [7/10], Train Loss: 0.1193, Val Acc: 96.25%
Epoch [8/10], Train Loss: 0.1167, Val Acc: 93.75%
Epoch [9/10], Train Loss: 0.1394, Val Acc: 93.75%
Epoch [10/10], Train Loss: 0.1179, Val Acc: 96.25%
Training complete.


In [10]:
torch.save(trained_model.state_dict(), 'resnet18_fire_trained.pth')

In [12]:
!pip install torchsummary

In [13]:
from torchsummary import summary

# Print model summary
summary(model, (3, 224, 224))  # Assuming the input size is 224x224

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [ ]:
!pip install fvcore

In [16]:
import torch
import time
from fvcore.nn import FlopCountAnalysis
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Ensure the model is on the correct device
model = models.resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

# Initialize a dummy input to calculate FLOPs
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# Compute FLOPs using fvcore
flops = FlopCountAnalysis(model, dummy_input)
print(f"FLOPs: {flops.total()}")

# Measure runtime on a batch of images
def measure_inference_time(model, data_loader):
    model.eval()
    start_time = time.time()
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            outputs = model(images)
            break  # Process just one batch
    end_time = time.time()
    avg_inference_time = (end_time - start_time) / len(data_loader.dataset)
    return avg_inference_time

# Assuming you have a DataLoader for your validation set
batch_size = 32
val_dataset = datasets.ImageFolder(root=wildfire_dataset_path, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Measure the inference time per image
avg_inference_time = measure_inference_time(model, val_loader)
print(f"Average inference time per image: {avg_inference_time:.4f} seconds")


FLOPs: 1826006016
Average inference time per image: 0.0014 seconds


## MobileNetV2 Model

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader
from torchsummary import summary

# Load the pretrained MobileNetV2 model
model = models.mobilenet_v2(pretrained=True)

# Adjust the final fully connected layer for binary classification (2 classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

# Move model to the available device (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 118MB/s]


In [18]:
def train_model(model, train_loader, val_loader, epochs=10, lr=0.001):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_acc = evaluate_model(model, val_loader)
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {running_loss/len(train_loader):.4f}, Val Acc: {val_acc:.2f}%")

    print("Training complete.")
    return model

def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

# Train the model
trained_model_mobilenet = train_model(model, fire_train_loader, fire_val_loader, epochs=10, lr=0.001)

# Save the trained model
torch.save(trained_model_mobilenet.state_dict(), 'mobilenetv2_fire_trained.pth')


Epoch [1/10], Train Loss: 0.1935, Val Acc: 100.00%
Epoch [2/10], Train Loss: 0.6964, Val Acc: 50.00%
Epoch [3/10], Train Loss: 0.5148, Val Acc: 50.00%
Epoch [4/10], Train Loss: 0.5106, Val Acc: 50.00%
Epoch [5/10], Train Loss: 0.5116, Val Acc: 50.00%
Epoch [6/10], Train Loss: 0.5143, Val Acc: 50.00%
Epoch [7/10], Train Loss: 0.5142, Val Acc: 50.00%
Epoch [8/10], Train Loss: 0.5109, Val Acc: 50.00%
Epoch [9/10], Train Loss: 0.5150, Val Acc: 50.00%
Epoch [10/10], Train Loss: 0.5127, Val Acc: 50.00%
Training complete.


Low accuracy due to imbalanced data. changing class weights and modifying code

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Calculate class weights
num_fire = 630
num_non_fire = 164
total_samples = num_fire + num_non_fire

# Weight for each class (higher weight for the minority class "non-fire")
fire_weight = total_samples / (2 * num_fire)  # For "fire" class
non_fire_weight = total_samples / (2 * num_non_fire)  # For "non-fire" class

# Assign weights to the classes
class_weights = torch.tensor([fire_weight, non_fire_weight], device=device)

# Define model, loss function with class weights, and optimizer
model = models.mobilenet_v2(weights="IMAGENET1K_V1")
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)  # Adjust for 2 classes
model = model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model as usual
trained_model = train_model(model, fire_train_loader, fire_val_loader, epochs=10, lr=0.001)


Epoch [1/10], Train Loss: 0.2566, Val Acc: 96.25%
Epoch [2/10], Train Loss: 0.9531, Val Acc: 80.00%
Epoch [3/10], Train Loss: 0.4370, Val Acc: 91.25%
Epoch [4/10], Train Loss: 0.3778, Val Acc: 85.00%
Epoch [5/10], Train Loss: 0.3499, Val Acc: 90.00%
Epoch [6/10], Train Loss: 0.2698, Val Acc: 86.25%
Epoch [7/10], Train Loss: 0.2858, Val Acc: 93.75%
Epoch [8/10], Train Loss: 0.5097, Val Acc: 61.25%
Epoch [9/10], Train Loss: 0.3635, Val Acc: 91.25%
Epoch [10/10], Train Loss: 0.2296, Val Acc: 95.00%
Training complete.


In [20]:
from torchsummary import summary

# Get summary of the trained model (assuming input size is 224x224 for MobileNetV2)
summary(trained_model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

In [22]:
from fvcore.nn import FlopCountAnalysis
import time
import torch
import torchvision.models as models

# Move model to eval mode and CPU for FLOPs calculation
trained_model.eval()
trained_model.cpu()

# Dummy input
dummy_input = torch.randn(1, 3, 224, 224)

# FLOPs calculation
flops = FlopCountAnalysis(trained_model, dummy_input)
print(f"FLOPs: {flops.total()}")

# Inference time calculation
trained_model.eval()
trained_model.to(device)
dummy_input = dummy_input.to(device)

with torch.no_grad():
    start_time = time.time()
    for _ in range(100):
        _ = trained_model(dummy_input)
    end_time = time.time()

avg_inference_time = (end_time - start_time) / 100
print(f"Average inference time per image: {avg_inference_time:.4f} seconds")

FLOPs: 312915776
Average inference time per image: 0.0058 seconds


## EfficientNetB0

In [23]:
from torchvision.models import efficientnet_b0

# Load pretrained EfficientNetB0
model = efficientnet_b0(weights="IMAGENET1K_V1")

# Modify final classifier layer for 2 classes
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

model = model.to(device)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 140MB/s]


In [24]:
# Reuse class weights from earlier
weights = torch.tensor([1.0, 630/164], dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)

In [25]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

trained_model = train_model(model, fire_train_loader, fire_val_loader, epochs=10, lr=0.001)
torch.save(trained_model.state_dict(), 'efficientnetb0_fire_trained.pth')

Epoch [1/10], Train Loss: 0.1938, Val Acc: 98.75%
Epoch [2/10], Train Loss: 155.6686, Val Acc: 52.50%
Epoch [3/10], Train Loss: 59.1824, Val Acc: 41.25%
Epoch [4/10], Train Loss: 48.3075, Val Acc: 50.00%
Epoch [5/10], Train Loss: 4.0257, Val Acc: 32.50%
Epoch [6/10], Train Loss: 0.8582, Val Acc: 50.00%
Epoch [7/10], Train Loss: 0.7048, Val Acc: 50.00%
Epoch [8/10], Train Loss: 0.6945, Val Acc: 50.00%
Epoch [9/10], Train Loss: 0.6938, Val Acc: 50.00%
Epoch [10/10], Train Loss: 0.6934, Val Acc: 50.00%
Training complete.


In [26]:
from torchsummary import summary
summary(trained_model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              SiLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              SiLU-6         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 112, 112]               0
           Conv2d-13         [-1, 16, 112, 112]             512
      BatchNorm2d-14         [-1, 16, 1

In [27]:
from fvcore.nn import FlopCountAnalysis
import time

dummy_input = torch.randn(1, 3, 224, 224).to(device)

# FLOPs
flops = FlopCountAnalysis(model, dummy_input)
print(f"FLOPs: {flops.total()}")

# Inference time
start_time = time.time()
with torch.no_grad():
    for _ in range(100):
        _ = model(dummy_input)
end_time = time.time()
print(f"Average inference time per image: {(end_time - start_time)/100:.4f} seconds")

features.1.0.stochastic_depth, features.2.0.stochastic_depth, features.2.1.stochastic_depth, features.3.0.stochastic_depth, features.3.1.stochastic_depth, features.4.0.stochastic_depth, features.4.1.stochastic_depth, features.4.2.stochastic_depth, features.5.0.stochastic_depth, features.5.1.stochastic_depth, features.5.2.stochastic_depth, features.6.0.stochastic_depth, features.6.1.stochastic_depth, features.6.2.stochastic_depth, features.6.3.stochastic_depth, features.7.0.stochastic_depth


FLOPs: 400381952
Average inference time per image: 0.0105 seconds
